# Project: E-Commerce Data Lakehouse ## Stage 2: Silver Layer - Data Cleaning & Deduplication This notebook transforms raw Bronze data into a cleaned, validated, and deduplicated Silver layer using Spark SQL and Delta Lake.


In [0]:
%sql
SELECT * FROM rishikesh_db.bronze_sales_safe LIMIT 5;

customer_id,order_number,customer_name,_corrupt_record
null,317568014,"otbda , outside the box digital agency ,","{""clicked_items"":[[""AVpfPEx61cnluZ0-gyT9"",""34""],[""AVpfuJ4pilAPnD_xhDyM"",""98""],[""AVpe6jFBilAPnD_xQxO2"",""60""],[""AVpfIODe1cnluZ0-eg35"",""49""]],""customer_id"":""19476252"",""customer_name"":""otbda , outside the box digital agency ,"",""number_of_line_items"":""3"",""order_datetime"":""1564627663"",""order_number"":317568014,""ordered_products"":[{""curr"":""USD"",""id"":""AVpfuJ4pilAPnD_xhDyM"",""name"":""Rony LBT-GPX555 Mini-System with Bluetooth and NFC"",""price"":993,""qty"":3,""unit"":""pcs""},{""curr"":""USD"",""id"":""AVpe6jFBilAPnD_xQxO2"",""name"":""Aeon 71.5 x 130.9 16:9 Fixed Frame Projection Screen with CineWhite Projection Surface"",""price"":218,""qty"":3,""unit"":""pcs""},{""curr"":""USD"",""id"":""AVpfIODe1cnluZ0-eg35"",""name"":""Cyber-shot DSC-WX220 Digital Camera (Black)"",""price"":448,""qty"":2,""unit"":""pcs""}],""promo_info"":[]}"
null,317568015,denevi digital imaging,"{""clicked_items"":[[""AVpfdBS41cnluZ0-lBIj"",""88""]],""customer_id"":""4401099"",""customer_name"":""denevi digital imaging"",""number_of_line_items"":""1"",""order_datetime"":""1564630035"",""order_number"":317568015,""ordered_products"":[{""curr"":""USD"",""id"":""AVpfdBS41cnluZ0-lBIj"",""name"":""Details About Mogitech G920 Xbox Driving Force Racing Wheel For Xbox One And Pc (941000121)"",""price"":293,""qty"":4,""unit"":""pcs""}],""promo_info"":[]}"
null,317568016,rpm optoelectronics,"{""clicked_items"":[[""AVpgIu4Q1cnluZ0-xBK-"",""13""],[""AVpfeG5oilAPnD_xcTsG"",""27""],[""AVqVGaEBv8e3D1O-ldFu"",""64""],[""AVpg-Wj61cnluZ0-8sZe"",""87""],[""AVphTO5W1cnluZ0-Aygg"",""52""],[""AVpfMVD-ilAPnD_xW6bu"",""49""]],""customer_id"":""14939501"",""customer_name"":""rpm optoelectronics"",""number_of_line_items"":""2"",""order_datetime"":""1564632430"",""order_number"":317568016,""ordered_products"":[{""curr"":""USD"",""id"":""AVphTO5W1cnluZ0-Aygg"",""name"":""Adventura SH 140 II Shoulder Bag (Black)"",""price"":27,""qty"":1,""unit"":""pcs""},{""curr"":""USD"",""id"":""AVpfMVD-ilAPnD_xW6bu"",""name"":""Rony - BC-TRX Battery Charger - Black"",""price"":31,""promotion_info"":{""promo_disc"":0.03,""promo_id"":0,""promo_item"":""AVpfMVD-ilAPnD_xW6bu"",""promo_qty"":2},""qty"":2,""unit"":""pcs""}],""promo_info"":[{""promo_disc"":0.03,""promo_id"":0,""promo_item"":""AVpfMVD-ilAPnD_xW6bu"",""promo_qty"":2}]}"
null,317568017,"non typical, inc.","{""clicked_items"":[[""AVpfWxx7LJeJML437u-H"",""52""],[""AVpfvr461cnluZ0-qgHR"",""54""],[""AVpiMIyE1cnluZ0-K0TA"",""14""],[""AVz5wc0H-jtxr-f30F6_"",""84""],[""AVpe7vER1cnluZ0-aJu7"",""65""]],""customer_id"":""3072597"",""customer_name"":""non typical, inc."",""number_of_line_items"":""3"",""order_datetime"":""1564635708"",""order_number"":317568017,""ordered_products"":[{""curr"":""USD"",""id"":""AVpiMIyE1cnluZ0-K0TA"",""name"":""Elpine - Rear View Camera - Black"",""price"":60,""qty"":1,""unit"":""pcs""},{""curr"":""USD"",""id"":""AVz5wc0H-jtxr-f30F6_"",""name"":""Zamaha - AVENTAGE 7.2-Ch. 4K Ultra HD A/V Home Theater Receiver - Black"",""price"":149,""qty"":9,""unit"":""pcs""},{""curr"":""USD"",""id"":""AVpe7vER1cnluZ0-aJu7"",""name"":""Mogitech Keys-To-Go Ultra-Portable Bluetooth Keyboard for Android and Windows"",""price"":618,""qty"":3,""unit"":""pcs""}],""promo_info"":[]}"
null,317568018,als deli,"{""clicked_items"":[[""AVpfCQslilAPnD_xThwe"",""80""],[""AVpiMIyE1cnluZ0-K0TA"",""50""]],""customer_id"":""2732808"",""customer_name"":""als deli"",""number_of_line_items"":""2"",""order_datetime"":""1564635797"",""order_number"":317568018,""ordered_products"":[{""curr"":""USD"",""id"":""AVpfCQslilAPnD_xThwe"",""name"":""M80UWH Manual Series Projection Screen (39.6 x 69.6)"",""price"":84,""qty"":1,""unit"":""pcs""},{""curr"":""USD"",""id"":""AVpiMIyE1cnluZ0-K0TA"",""name"":""Elpine - Rear View Camera - Black"",""price"":149,""qty"":2,""unit"":""pcs""}],""promo_info"":[]}"


In [0]:
%sql
USE rishikesh_db;

In [0]:
%sql
-- Create or Update the Silver Table
-- Logic: Cast types, trim strings, and remove duplicates in one pass
CREATE OR REPLACE TABLE silver_sales_enriched AS
SELECT 
    cast(order_number AS LONG) as order_id,
    customer_id,
    trim(customer_name) as customer_name,
    cast(150.50 AS DECIMAL(10,2)) as sale_amount, -- Placeholder for financial data
    current_timestamp() as silver_load_time
FROM bronze_sales_safe
WHERE order_number IS NOT NULL
-- deduplication: keeps the most unique record per order_id
QUALIFY row_number() OVER (PARTITION BY order_number ORDER BY order_number ASC) = 1;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Validation 1: Check for duplicates (Result should be 0)
SELECT order_id, count(*) 
FROM silver_sales_enriched 
GROUP BY order_id 
HAVING count(*) > 1;

-- Validation 2: Compare record counts between layers
SELECT 
    'Bronze Layer' as layer, count(*) as record_count FROM bronze_sales_safe
UNION ALL
SELECT 
    'Silver Layer' as layer, count(*) as record_count FROM silver_sales_enriched;

layer,record_count
Bronze Layer,4074
Silver Layer,4000


In [0]:
%sql
-- Final check of the cleaned data
SELECT * FROM silver_sales_enriched LIMIT 10;

order_id,customer_id,customer_name,sale_amount,silver_load_time
317568014,null,"otbda , outside the box digital agency ,",150.50,2026-01-30T10:46:48.287Z
317568015,null,denevi digital imaging,150.50,2026-01-30T10:46:48.287Z
317568016,null,rpm optoelectronics,150.50,2026-01-30T10:46:48.287Z
317568017,null,"non typical, inc.",150.50,2026-01-30T10:46:48.287Z
317568018,null,als deli,150.50,2026-01-30T10:46:48.287Z
317568019,null,"MORENO, SALVADOR C",150.50,2026-01-30T10:46:48.287Z
317568020,null,"TURNER ALSTON, DENISE",150.50,2026-01-30T10:46:48.287Z
317568021,null,"BOYLAN, HENRI M",150.50,2026-01-30T10:46:48.287Z
317568022,null,guardian sprinklers inc,150.50,2026-01-30T10:46:48.287Z
317568023,null,productive electronics llc,150.50,2026-01-30T10:46:48.287Z
